In [26]:
import os
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
# Function to preprocess images: grayscale + 64x64 resize
def preprocess_image(image_path, target_size=(64, 64)):
    img = cv2.imread(image_path)  
    resized_img = cv2.resize(img, target_size)  # Resize to 64x64
    return resized_img.flatten()  # Flatten the image for easier indexing

# Preprocess all images in the dataset
def preprocess_dataset(image_folder, target_size=(64, 64), max_workers=8):  # Set to 8 for your CPU
    processed_images = []
    image_paths = []
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('jpg', 'jpeg', 'png'))]
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(preprocess_image, os.path.join(image_folder, img_file), target_size): img_file for img_file in image_files}
        
        for future in futures:
            img_file = futures[future]
            try:
                processed_img = future.result()
                if processed_img is not None:  # Check if the image was processed successfully
                    processed_images.append(processed_img)
                    image_paths.append(os.path.join(image_folder, img_file))
                    print(f"Processed {img_file}")
            except Exception as e:
                print(f"Error processing {img_file}: {e}")
    return np.array(processed_images), image_paths

# Example usage:
image_folder = 'C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg'
processed_images, image_paths = preprocess_dataset(image_folder)

Processed 20056.jpg
Processed 20057.jpg
Processed 20058.jpg
Processed 20059.jpg
Processed 20060.jpg
Processed 20061.jpg
Processed 20062.jpg
Processed 20064.jpg
Processed 20066.jpg
Processed 20067.jpg
Processed 20068.jpg
Processed 20069.jpg
Processed 20070.jpg
Processed 20071.jpg
Processed 20072.jpg
Processed 20073.jpg
Processed 20074.jpg
Processed 20075.jpg
Processed 20076.jpg
Processed 20077.jpg
Processed 20078.jpg
Processed 20079.jpg
Processed 20080.jpg
Processed 20081.jpg
Processed 20082.jpg
Processed 20083.jpg
Processed 20084.jpg
Processed 20085.jpg
Processed 20087.jpg
Processed 20088.jpg
Processed 20089.jpg
Processed 20090.jpg
Processed 20091.jpg
Processed 20092.jpg
Processed 20093.jpg
Processed 20094.jpg
Processed 20095.jpg
Processed 20096.jpg
Processed 20098.jpg
Processed 20099.jpg
Processed 20100.jpg
Processed 20106.jpg
Processed 20107.jpg
Processed 20108.jpg
Processed 20109.jpg
Processed 20111.jpg
Processed 20112.jpg
Processed 20113.jpg
Processed 20114.jpg
Processed 20115.jpg


In [27]:
from sklearn.decomposition import PCA

# Reduce dimensionality using PCA
def reduce_dimensionality(images, n_components=50):
    pca = PCA(n_components=n_components)
    reduced_images = pca.fit_transform(images)  # Reduce dimensions
    return reduced_images, pca

# Example usage:
reduced_images, pca_model = reduce_dimensionality(processed_images)


In [28]:
!pip install faiss-cpu


In [29]:
import faiss

# Build FAISS index
def build_faiss_index(image_vectors):
    dimension = image_vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean distance)
    index.add(image_vectors)  # Add the dataset vectors to the index
    return index

# Example usage:
faiss_index = build_faiss_index(reduced_images)


In [30]:
# Function to search for the closest match

from tensorflow.keras.models import load_model
model = load_model('C:/Users/deepa/OneDrive/Desktop/STGI02/tests/ai_image_detection_model.h5')

def search_image(query_image_processed, index, pca_model, top_k=5):
    # Apply PCA to query image (which is already preprocessed)
    reduced_query_image = pca_model.transform([query_image_processed])  # Apply PCA to query image
    
    # Search FAISS index for top-k closest images
    distances, indices = index.search(np.array(reduced_query_image).astype('float32'), top_k)
    
    return indices[0], distances[0]

# Function to preprocess the query image (grayscale, resize, flatten)
def preprocess_query_image(query_image_path, target_size=(64, 64)):
    img = cv2.imread(query_image_path)  # Read image in color (BGR)
    if img is None:
        raise ValueError(f"Image not found or invalid path: {query_image_path}")

    resized_img = cv2.resize(img, target_size)  # Resize to 64x64
    return resized_img  # Normalize to [0, 1]

# Test

def predict_ai_generated(image_path):
    # Preprocess the image
    new_image = preprocess_query_image(image_path)

    # Check the shape of the image before prediction
    print(f"Preprocessed image shape: {new_image.shape}")  # Should be (64, 64, 3)

    # Make the prediction; expand dimensions to match model input shape
    new_image = np.expand_dims(new_image, axis=0)  # Add batch dimension
    print(f"Shape after adding batch dimension: {new_image.shape}")  # Should be (1, 64, 64, 3)

    # Make the prediction
    prediction = model.predict(new_image)

    # Determine if it's AI-generated based on the prediction threshold
    is_ai_generated = prediction[0][0] > 0.5  # Adjust threshold if necessary

    return is_ai_generated



uploaded_image_path = "C:/Users/deepa/OneDrive/Desktop/STGI02/085373.jpg"  # Path to the uploaded image

is_ai_image = predict_ai_generated(uploaded_image_path)
print(f"The image is {'AI-generated' if is_ai_image else 'not AI-generated'}.")

# Process the query image for searching
query_image_processed = preprocess_query_image(uploaded_image_path, target_size=(64, 64)) 
query_image_processed_flat = query_image_processed.flatten()  # Flatten to 1D array for PCA

# Search for closest images
closest_indices, distances = search_image(query_image_processed_flat, faiss_index, pca_model)

# Fetch closest images based on indices
closest_images = [image_paths[idx] for idx in closest_indices]
print(f"Closest matches: {closest_images}")
print(f"Similarities: {1/(1 + distances)}")
print(f"Distances: {distances}")

Preprocessed image shape: (64, 64, 3)
Shape after adding batch dimension: (1, 64, 64, 3)
1/1 [==============================] - 1s 1s/step
The image is not AI-generated.
Closest matches: ['C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg\\22978.jpg', 'C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg\\21607.jpg', 'C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg\\20759.jpg', 'C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg\\23179.jpg', 'C:/Users/deepa/OneDrive/Desktop/STGI02/Randomimg\\23757.jpg']
Similarities: [2.4591607e-08 2.2722196e-08 2.1634932e-08 2.1184009e-08 2.0502590e-08]
Distances: [40664280. 44009832. 46221544. 47205416. 48774324.]
